In [1]:
import subprocess
import google.auth
import json
import traceback

In [2]:
glb_errors_list = []

In [3]:
result = subprocess.run(['gcloud', 'projects', 'list', '--format', 'json', '--quiet' ], shell = True, capture_output=True, text=True, check=True )

In [4]:
projects = json.loads(result.stdout)

In [5]:
def delete_compute(project_id):
    from google.cloud import compute_v1
    from collections import defaultdict

    instance_client = compute_v1.InstancesClient()

    print(f"Getting list of all compute instances for project - {project_id}")

    request = compute_v1.AggregatedListInstancesRequest()
    request.project = project_id
    # Use the `max_results` parameter to limit the number of results that the API returns per response page.
    request.max_results = 50

    agg_list = instance_client.aggregated_list(request=request)

    all_instances = defaultdict(list)

    # Despite using the `max_results` parameter, you don't need to handle the pagination
    # yourself. The returned `AggregatedListPager` object handles pagination
    # automatically, returning separated pages as you iterate over the results.
    for zone, response in agg_list:
        if response.instances:
            all_instances[zone].extend(response.instances)
            print(f" {zone}:")
            for instance in response.instances:
                try:
                    print(f" - {instance.name} ({instance.machine_type})")
                    instance.delete_compute()
                    print(f"Instance deleted- {instance.name}")
                except:
                    #print(traceback.format_exc())
                    global glb_errors_list
                    glb_errors_list.append(f"{project_id}, instance.name")
                    print(f"Error: {project_id}:: {instance.name}")
    return all_instances

In [6]:
def delete_gke(PROJECT_ID):
    print(f"Deleting gke for project - {PROJECT_ID}")
    results = subprocess.run(['gcloud', 'container', 'clusters', 'list', '--format', 'json', '--quiet' ], shell = True, capture_output=True, text=True, check=True )
    for result in json.loads(results.stdout):
        NAME = result.get("name")
        ZONE = result.get("zone")

        try:
            results_disk = subprocess.run(['gcloud', 'container', 'clusters', 'delete', NAME, '--zone', ZONE, '--project', PROJECT_ID, '--quiet' ], shell = True, capture_output=True, text=True, check=True )
            print(f"GKE deleted- {NAME}")
        except:
            #print(traceback.format_exc())
            global glb_errors_list
            glb_errors_list.append(f"{project_id}, instance.name")
            print(f"Error: {PROJECT_ID}:: {NAME}")

In [7]:
def delete_sql_cluster(PROJECT_ID):
    print(f"Deleting SQL Clusters for project - {PROJECT_ID}")
    results = subprocess.run(['gcloud', 'sql', 'instances', 'list', '--format', 'json', '--quiet' ], shell = True, capture_output=True, text=True, check=True )
    for result in json.loads(results.stdout):
        NAME = result.get("name")

        try:
            print(f"Deleting SQL Cluster - {NAME}")
            results_disk = subprocess.run(['gcloud', 'sql', 'instances', 'delete', NAME, '--project', PROJECT_ID, '--quiet' ], shell = True, capture_output=True, text=True, check=True )
            print(f"SQL Cluster deleted- {NAME}")
        except:
            #print(traceback.format_exc())
            global glb_errors_list
            glb_errors_list.append(f"{project_id}, instance.name")
            print(f"Error: {PROJECT_ID}:: {NAME}")

In [8]:
def delete_unused_disks(PROJECT_ID):
    print(f"Deleting disks for project - {PROJECT_ID}")
    results = subprocess.run(['gcloud', 'compute', 'disks', 'list', '--format', 'json', '--quiet' ], shell = True, capture_output=True, text=True, check=True )
    for result in json.loads(results.stdout):
        NAME = result.get("name")
        ZONE = result.get("zone")

        try:
            results_disk = subprocess.run(['gcloud', 'compute', 'disks', 'delete', NAME, '--zone', ZONE, '--project', PROJECT_ID, '--quiet' ], shell = True, capture_output=True, text=True, check=True )
            print(f"disk deleted- {NAME}")
        except:
            #print(traceback.format_exc())
            global glb_errors_list
            glb_errors_list.append(f"{project_id}, instance.name")
            print(f"Error: {PROJECT_ID}:: {NAME}")

In [9]:
def delete_buckets(PROJECT_ID):
    from google.cloud import storage
    
    gcs = storage.Client(project = PROJECT_ID)

    print(f"Getting list of all buckets for project - {PROJECT_ID}")
    buckets = gcs.list_buckets()

    
    for ktr, bucket in enumerate(buckets):
        try:
            print(f"{ktr} Delete Bucket. PROJECT_ID: {PROJECT_ID}, bucket.name: {bucket.name}")
            bucket.delete(force=True)
            print(f"Bucket deleted- {bucket.name}")
        except:
            #print(traceback.format_exc())
            global glb_errors_list
            glb_errors_list.append(f"{project_id}, instance.name")
            print(f"Error: {PROJECT_ID}:: {bucket.name}")

In [10]:
def delete_resources(PROJECT_ID):
    from google.cloud import storage
    from google.cloud import bigquery
    
    
    delete_sql_cluster(PROJECT_ID)
    #delete_gke(project_id)
    #delete_buckets(PROJECT_ID)
    #delete_compute(PROJECT_ID)
    #delete_unused_disks(PROJECT_ID)
    

In [11]:
for  project in projects:
    try:
        project_id = project.get("projectId")
        project_name = project.get("name")
        project_no = project.get("projectNumber")
        subprocess.run(['gcloud', 'config', 'set', 'project', project_id, ], shell = True )

        print ("Working on ", project_name)
        
        delete_resources(project_id)
        print(glb_errors_list)
    except:
        print(traceback.format_exc())

Working on  GCP-LAB-29
Deleting SQL Clusters for project - gcp-lab-29
[]
Working on  GCP-LAB-28
Deleting SQL Clusters for project - gcp-lab-28
[]
Working on  GCP-LAB-27
Deleting SQL Clusters for project - gcp-lab-27
[]
Working on  GCP-LAB-26
Deleting SQL Clusters for project - gcp-lab-26
Traceback (most recent call last):
  File "C:\Users\atingupta2005\AppData\Local\Temp\ipykernel_22736\3438781852.py", line 10, in <module>
    delete_resources(project_id)
  File "C:\Users\atingupta2005\AppData\Local\Temp\ipykernel_22736\1650821834.py", line 6, in delete_resources
    delete_sql_cluster(PROJECT_ID)
  File "C:\Users\atingupta2005\AppData\Local\Temp\ipykernel_22736\3147105774.py", line 3, in delete_sql_cluster
    results = subprocess.run(['gcloud', 'sql', 'instances', 'list', '--format', 'json', '--quiet' ], shell = True, capture_output=True, text=True, check=True )
  File "C:\Program Files\Python39\lib\subprocess.py", line 528, in run
    raise CalledProcessError(retcode, process.args,


In [12]:
glb_errors_list

[]